In [60]:
import pandas as pd
import numpy as np
import os
import glob
os.chdir("./")
import numpy as np

In [61]:
def distance_calc(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    mi = 3956.27 * c
    return mi

In [62]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [63]:
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])
combined_csv.to_csv( "combined_JC-202102-202109-citibike-tripdata.csv", index=False, encoding='utf-8-sig')

In [64]:
data = pd.read_csv('combined_JC-202102-202109-citibike-tripdata.csv')
data

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,121DD7DD23CB1335,docked_bike,2021-02-03 23:11:28,2021-02-03 23:18:28,Hoboken Ave at Monmouth St,JC105,Christ Hospital,JC034,40.735208,-74.046964,40.734786,-74.050444,member
1,FD73FB85F008349D,docked_bike,2021-02-27 16:34:05,2021-02-27 16:56:40,Newport Pkwy,JC008,Marin Light Rail,JC013,40.728744,-74.032108,40.714584,-74.042817,member
2,39F9E6663CB5FDF6,docked_bike,2021-02-26 23:16:04,2021-02-26 23:22:25,Journal Square,JC103,Baldwin at Montgomery,JC020,40.733670,-74.062500,40.723659,-74.064194,member
3,A64745CB0792EC6F,docked_bike,2021-02-24 16:51:50,2021-02-24 17:16:09,Hoboken Ave at Monmouth St,JC105,Hoboken Ave at Monmouth St,JC105,40.735208,-74.046963,40.735208,-74.046964,casual
4,75CC76EB9543764A,docked_bike,2021-02-24 20:44:16,2021-02-24 20:44:46,Hoboken Ave at Monmouth St,JC105,Hoboken Ave at Monmouth St,JC105,40.735208,-74.046963,40.735208,-74.046964,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
435883,DA808C4DE967D604,classic_bike,2021-09-23 19:08:20,2021-09-23 19:44:02,Harborside,JC104,Union St,JC051,40.719252,-74.034234,40.718211,-74.083639,casual
435884,667A3B00C664DBAF,classic_bike,2021-09-10 15:47:24,2021-09-10 16:36:17,8 St & Washington St,HB603,Union St,JC051,40.745984,-74.028199,40.718211,-74.083639,casual
435885,62E1888ABDE1464E,classic_bike,2021-09-30 13:28:29,2021-09-30 13:54:09,Harborside,JC104,Union St,JC051,40.719252,-74.034234,40.718211,-74.083639,member
435886,475AA9DFCF196D50,classic_bike,2021-09-14 03:31:22,2021-09-14 04:18:35,Harborside,JC104,Union St,JC051,40.719252,-74.034234,40.718211,-74.083639,casual


In [65]:
data.started_at = pd.to_datetime(data.started_at)
data.ended_at = pd.to_datetime(data.ended_at)

In [66]:
timedelta = []
ssn_return = []

for index, row in data.iterrows():
    timedelta.append((row.ended_at - row.started_at).seconds / 60.0)
    if row.start_station_id == row.end_station_id:
        ssn_return.append('Yes')
    else:
        ssn_return.append('No')

data['duration_in_minutes'] = timedelta
data['same_station_return'] = ssn_return

In [67]:
data['distance_in_miles'] = distance_calc(data['start_lng'],data['start_lat'],data['end_lng'],data['end_lat'])

In [68]:
data = data.drop(['ride_id','start_station_id','end_station_id'], axis=1)
data

,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,duration_in_minutes,same_station_return,distance_in_miles
0,docked_bike,2021-02-03 23:11:28,2021-02-03 23:18:28,Hoboken Ave at Monmouth St,Christ Hospital,40.735208,-74.046964,40.734786,-74.050444,member,7.000000,No,0.184396
1,docked_bike,2021-02-27 16:34:05,2021-02-27 16:56:40,Newport Pkwy,Marin Light Rail,40.728744,-74.032108,40.714584,-74.042817,member,22.583333,No,1.126971
2,docked_bike,2021-02-26 23:16:04,2021-02-26 23:22:25,Journal Square,Baldwin at Montgomery,40.733670,-74.062500,40.723659,-74.064194,member,6.350000,No,0.696928
3,docked_bike,2021-02-24 16:51:50,2021-02-24 17:16:09,Hoboken Ave at Monmouth St,Hoboken Ave at Monmouth St,40.735208,-74.046963,40.735208,-74.046964,casual,24.316667,Yes,0.000047
4,docked_bike,2021-02-24 20:44:16,2021-02-24 20:44:46,Hoboken Ave at Monmouth St,Hoboken Ave at Monmouth St,40.735208,-74.046963,40.735208,-74.046964,member,0.500000,Yes,0.000047
...,...,...,...,...,...,...,...,...,...,...,...,...,...
435883,classic_bike,2021-09-23 19:08:20,2021-09-23 19:44:02,Harborside,Union St,40.719252,-74.034234,40.718211,-74.083639,casual,35.700000,No,2.586600
435884,classic_bike,2021-09-10 15:47:24,2021-09-10 16:36:17,8 St & Washington St,Union St,40.745984,-74.028199,40.718211,-74.083639,casual,48.883333,No,3.477431
435885,classic_bike,2021-09-30 13:28:29,2021-09-30 13:54:09,Harborside,Union St,40.719252,-74.034234,40.718211,-74.083639,member,25.666667,No,2.586600
435886,classic_bike,2021-09-14 03:31:22,2021-09-14 04:18:35,Harborside,Union St,40.719252,-74.034234,40.718211,-74.083639,casual,47.216667,No,2.586600


In [69]:
data.to_csv('cleaned_JC-202102-202109-citibike-tripdata.csv')